<font face = "Times New Roman">

# ***Volume per dealer notebook*** 

### ***This notebook is used for checking dealers volume split for a big order*** 


<font>

In [1]:
import pandas as pd
import numpy as np
import importlib
import datetime as dt
import requests
import sys
import os
import logging
import time

sys.path.append(os.environ["HOME"]+"/trading/python/lib")
import talos_utils

sys.path.append(os.environ["HOME"]+"/trading/keys/")
import keys

ModuleNotFoundError: No module named 'keys'

In [ ]:
# Talos client init

# HOLD env 
talos_api = keys.talos_api
talos_secret_api = keys.talos_secret_api
host = keys.talos_host
talos = talos_utils.Talos(talos_api, talos_secret_api, host)

### wl env
talos_adv_api = keys.talos_api_key_wl
talos_adv_secret_api = keys.talos_api_secret_wl
host_wl = keys.talos_host_wl
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)

<font face = "Times New Roman">


### ***Input*** 

#### 1. Order ID List 
#### 2. Cutoff time for numbers checking. This is in Easter timezone
#### 3. Talos env, "a1" or "hold"  


<font>

In [2]:
all_orders = ['304a3f4f-ffed-4fe3-b420-e1fc6a6c39ae'] # input all order ID into the list, use Microstrategy as an example here
dealer_cutoff_start = dt.datetime(2024, 1, 17, 9, 0, 0, 0)  # in EST
dealer_cutoff_end = dt.datetime(2025, 1, 21, 9, 0, 0, 0)  # in EST
env = "hold"   # hold or "a1"``

In [3]:
def multi_order_fills(all_orders,talos_env):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos_env.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    return combined_df

def fills_by_dealer(df, dealer_cutoff_start=None, dealer_cutoff_end=None):
 
    if dealer_cutoff_start and dealer_cutoff_end is not None:
        filtered_df = df.loc[(df['Timestamp'] >= dealer_cutoff_start)
                             & (df['Timestamp'] < dealer_cutoff_end)]
    else:
        filtered_df = df
    # Group by dealer
    df_by_dealer = filtered_df.groupby("Market")[["amount_less_fees", "Quantity"]].sum().reset_index()
    df_count = filtered_df.groupby("Market")["OrderID"].count().reset_index()
    df_count.rename(columns={"OrderID": "fill_count"})
    
    pd.options.display.float_format = '{:,.8f}'.format
    amount = filtered_df['amount_less_fees'].sum()
    qty = filtered_df['Quantity'].sum() 

    summary_str = f"Amount: ${amount:,.4f} \nQuantity: {qty:,.4f}\n"
    summary_df = df_by_dealer.merge(df_count)
    return filtered_df, summary_str, summary_df

In [5]:
talos_env = talos if env == "hold" else talos_wl
combined_df = multi_order_fills(all_orders,talos_env)
filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start, dealer_cutoff_end)
print(summary_str)
summary_df = summary_df.set_index('Market')
summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
summary_df.loc["TOTAL"] = summary_df.sum()
summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

summary_df

NameError: name 'talos' is not defined

In [6]:
summary_df.sum()

NameError: name 'summary_df' is not defined

MSTR

In [7]:
from google.cloud import bigquery
import pandas as pd

bqclient = bigquery.Client(project="production-191601")

MSTR = """
SELECT
    organizations.org_name AS organization_name,
    trading_client_trades.*
FROM `production-191601.sales.salesforce_user` AS salesforce_user
LEFT JOIN `production-191601.sales.salesforce_account` AS salesforce_account
    ON salesforce_account.owner_id = salesforce_user.id
FULL OUTER JOIN `brokerage.client_trades` AS trading_client_trades
    ON salesforce_account.org_id_c = trading_client_trades.organization_key_id
LEFT JOIN `client_operations.organizations_confidential` AS organizations
    ON trading_client_trades.organization_key_id = organizations.org_id
WHERE
    organizations.org_name = 'MicroStrategy'
    AND trading_client_trades.created_at >= TIMESTAMP('2023-01-01 00:00:00')
    AND trading_client_trades.created_at < CURRENT_TIMESTAMP()
    AND trading_client_trades.latest_status IN ('EXECUTED', 'SETTLED', 'SETTLING')
    AND (
        shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) NOT IN ('USDC-USD', 'USD-USDC', 'PYUSD-USD')
        OR shared.ASSET_SYMBOL_MAP(trading_client_trades.symbol) IS NULL
    )
ORDER BY
    trading_client_trades.created_at DESC;

"""

trades_mstr = bqclient.query(MSTR).to_dataframe()

id_list = trades_mstr['id'].tolist()
filtered_dealer_trades_query = f"""
SELECT *
FROM brokerage.dealer_trades_view
WHERE symbol = 'BTC-USD'
"""

# Execute the query to filter dealer trades
filtered_dealer_trades = bqclient.query(filtered_dealer_trades_query).to_dataframe()
filtered_dealer_trades = filtered_dealer_trades[filtered_dealer_trades['parent_order_id'].isin(id_list)]



/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:207: UserWarning: Unable to determine type for field 'commissions'.
  warnings.warn("Unable to determine type for field '{}'.".format(bq_field.name))


In [8]:
# Create a list of order_ids from the filtered_dealer_trades DataFrame
order_id_list = filtered_dealer_trades['order_id'].tolist()

# Now order_id_list contains all the order_id values
print(order_id_list)


['4a849032-fdf1-4178-8bbc-0605be3a43ca', 'e9362735-6b03-4b59-9e4d-fa223e87fcb0', 'fb3650e7-5179-49b9-bf15-d0d2cb25a78f', 'b73e3f96-d064-40fa-8009-ef1a036a6270', '632f3014-b85e-4aa5-ac58-21aad22d9346', '0c990c5a-7b35-4b14-8717-d74a0af7afcf', '58c3ed85-6114-468a-ac34-8788143db2a1', '0fab908e-2b4f-468b-8eb3-be129f6e5068', '9cadde24-81cd-4460-ba95-5ac9f14af440', '8680dafa-7b53-4b07-abbd-9f3ebe3d3914', '118f12a5-636b-4576-8e43-4f455e82bf05', '0eb3e446-d003-4178-a1bc-3cdcfc80393f', 'db0fca76-786d-44e4-a90c-1e026acabf70', '8a770c09-0877-4aa1-87d3-e9d73a89f42e', 'cf8d4616-8356-47c7-aa51-a83f9e1b9bed', '5c30c215-ccea-4017-97ae-195622c45d56', '77d47686-a2a6-4e70-9c46-a2a07ced4618', '462084d7-8743-4387-b032-03b1c3b7d7f6', '50e4c912-cfa4-449a-b316-e2e2748770b0', '3c85488c-19b6-4699-a57a-8787f49caf54', '08b7caa0-acb4-4016-a535-3dd5bfc47582', '2f24bc27-9bc5-4bda-8bf9-8d0ca5048562', 'f9627007-f3fa-4e12-a393-316d4f60e9e9', 'cbfd2cc0-4463-4e71-85a5-6b73d9dea0d9', 'debd419a-368d-4aaa-b527-847c5c665f17',

In [9]:
def process_single_order(order_id, talos_env):
    results = {}  # To store the result for this single order_id

    print(f"Processing order ID: {order_id}")
        
    # Step 1: Get the combined data for the order
    combined_df = multi_order_fills([order_id], talos_env)
    print("combined_df:")
    print(combined_df)

    # Inspect column names to find where the Date/Time is stored
    print("Columns in combined_df:", combined_df.columns)

    # Step 2: Apply fills_by_dealer to get the summary
    filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start=None, dealer_cutoff_end=None)
    
    # Step 3: Set index and add win_rate column
    summary_df = summary_df.set_index('Market')
    summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
    summary_df.loc["TOTAL"] = summary_df.sum()  # Add a row for TOTAL
    summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

    # Step 4: Extract the first date from the 'TransactTime' column and use the first 10 characters
    if 'TransactTime' in combined_df.columns:
        order_date = combined_df['TransactTime'].iloc[0][:10]  # Get the first 10 characters (yyyy-mm-dd)
    else:
        order_date = None

    # Step 5: Convert the dataframe into a dictionary format
    market_data_dict = summary_df.to_dict(orient="index")  # Converts each row into a dictionary keyed by Market
        
    # Store the result for this order_id, including the date
    results[order_id] = {
        "summary_str": summary_str,  # The summary string (Amount, Quantity, etc.)
        "market_data": market_data_dict,  # The market data as a dictionary
        "Date": order_date  # Add the Date of the order (yyyy-mm-dd format)
    }

    return results

# Run the function to process just one order
order_id = order_id_list[0]  # Take the first order in the list (or just define a single order)
results = process_single_order(order_id, talos_env)

# Print the results for the single order
print("Results for order ID:", order_id)
print(results[order_id])


NameError: name 'talos_env' is not defined

In [ ]:
data = []

# Iterate over each order_id and its corresponding data
for order_id, order_info in results.items():
    order_date = order_info['Date']
    market_data = order_info['market_data']

    # Iterate through each market (dealer)
    for market, market_info in market_data.items():
        # Prepare the row with the date and the quantity for each dealer
        row = {
            'Date': order_date,
            'Market': market,
            'Quantity': market_info['quantity']
        }
        data.append(row)

# Create the DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame to have Date as the index and Markets as columns
df_pivot = df.pivot(index='Date', columns='Market', values='Quantity')

# Fill any missing data with 0 (if you want it to be NaN, you can omit this step)
df_pivot = df_pivot.fillna(0)
df_pivot

Market,TOTAL,cumberland,janestreet,wintermute
Date,,,,
2024-08-30,85.02802013,2.10000000,70.18600000,12.74202013


In [ ]:
def process_all_orders(order_id_list, talos_env):
    results = {}  # To store the results for all order_ids

    # Loop over each order_id in the list
    for order_id in order_id_list:
        print(f"Processing order ID: {order_id}")
        
        # Step 1: Get the combined data for the order
        combined_df = multi_order_fills([order_id], talos_env)
        print("combined_df:")
        print(combined_df)

        # Inspect column names to find where the Date/Time is stored
        print("Columns in combined_df:", combined_df.columns)

        # Step 2: Apply fills_by_dealer to get the summary
        filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start=None, dealer_cutoff_end=None)
        
        # Step 3: Set index and add win_rate column
        summary_df = summary_df.set_index('Market')
        summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
        summary_df.loc["TOTAL"] = summary_df.sum()  # Add a row for TOTAL
        summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

        # Step 4: Extract the first date from the 'TransactTime' column and use the first 10 characters
        if 'TransactTime' in combined_df.columns:
            order_date = combined_df['TransactTime'].iloc[0][:10]  # Get the first 10 characters (yyyy-mm-dd)
        else:
            order_date = None

        # Step 5: Convert the dataframe into a dictionary format
        market_data_dict = summary_df.to_dict(orient="index")  # Converts each row into a dictionary keyed by Market
            
        # Store the result for this order_id, including the date
        results[order_id] = {
            "summary_str": summary_str,  # The summary string (Amount, Quantity, etc.)
            "market_data": market_data_dict,  # The market data as a dictionary
            "Date": order_date  # Add the Date of the order (yyyy-mm-dd format)
        }

    return results

# Run the function to process all order_ids
results = process_all_orders(order_id_list, talos_env)

# Print the results for each order_id
for order_id, data in results.items():
    print(f"Results for order ID {order_id}:")
    print(data)


Processing order ID: 304a3f4f-ffed-4fe3-b420-e1fc6a6c39ae
combined_df:
                     Timestamp            User   Symbol   
0   2024-08-30 16:01:32.786555  Michael Marano  BTC-USD  \
1   2024-08-30 16:01:32.812541  Michael Marano  BTC-USD   
2   2024-08-30 16:01:32.812620  Michael Marano  BTC-USD   
3   2024-08-30 16:01:32.837626  Michael Marano  BTC-USD   
4   2024-08-30 16:01:32.863698  Michael Marano  BTC-USD   
..                         ...             ...      ...   
403 2024-08-30 17:33:36.097582  Michael Marano  BTC-USD   
404 2024-08-30 17:33:36.201451  Michael Marano  BTC-USD   
405 2024-08-30 17:33:37.171171  Michael Marano  BTC-USD   
406 2024-08-30 17:33:37.197865  Michael Marano  BTC-USD   
407 2024-08-30 17:33:37.222269  Michael Marano  BTC-USD   

                                  OrderID   
0    304a3f4f-ffed-4fe3-b420-e1fc6a6c39ae  \
1    304a3f4f-ffed-4fe3-b420-e1fc6a6c39ae   
2    304a3f4f-ffed-4fe3-b420-e1fc6a6c39ae   
3    304a3f4f-ffed-4fe3-b420-e1fc6a6c39a

In [ ]:
def create_dealer_quantity_df(results):
    # Initialize an empty list to store the data for the DataFrame
    data = []

    # Loop through each order_id in the results
    for order_id, order_info in results.items():
        order_date = order_info['Date']
        market_data = order_info['market_data']

        # Loop through each dealer (market)
        for market, market_info in market_data.items():
            # Prepare the row with the date, market, and quantity for each dealer
            row = {
                'Date': order_date,
                'Market': market,
                'Quantity': market_info['quantity']
            }
            data.append(row)

    # Create the DataFrame
    df = pd.DataFrame(data)

    # Use pivot_table to handle duplicates and sum the Quantity for each Date and Market
    df_pivot = df.pivot_table(index='Date', columns='Market', values='Quantity', aggfunc='sum')

    # Fill any missing data with 0 (if you want it to be NaN, you can omit this step)
    df_pivot = df_pivot.fillna(0)

    return df_pivot

# Run the function to create the DataFrame for all orders
df_all_orders = create_dealer_quantity_df(results)
df_all_orders = df_all_orders.sort_index(ascending=False)

# Print the result
df_all_orders


Market,TOTAL,cumberland,flowtraders,galaxy,janestreet,nonco,wintermute
Date,,,,,,,
2025-01-24,262.57306646,25.97820939,0.00000000,79.50990689,52.69999161,0.00000000,104.38495857
2025-01-23,"1,196.41448200",89.39131209,0.00000000,462.74309609,556.23127880,0.00000000,88.04879502
2025-01-21,"1,998.57352826",305.29615573,0.00000000,681.49790474,903.52996792,0.00000000,108.24949987
2025-01-17,"1,432.06754083",87.38676126,0.00000000,567.18447725,623.64021499,0.00000000,153.85608733
2025-01-16,705.72363709,126.61448829,0.00000000,153.49522106,396.52320736,0.00000000,29.09072038
...,...,...,...,...,...,...,...
2024-03-09,"1,799.84493772",47.98401770,0.00000000,0.00000000,871.04471521,0.00000000,880.81620481
2024-03-08,"1,802.58917796",27.77696692,0.00000000,0.00000000,"1,229.83220046",0.00000000,544.98001058
2024-02-16,519.49127979,47.07558129,0.00000000,0.00000000,297.84840568,0.00000000,174.56729282
